# 모델

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

In [9]:
# 라이브러리

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import random
from cv2 import resize
from glob import glob
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input


In [6]:
# 이미지 데이터셋 로드 및 전처리

img_height = 244
img_width = 244
num_classes = 10

train_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/drive/MyDrive/1주일 프로젝트/Data_set',
  validation_split=0.2,
  subset='training',
  image_size=(img_height, img_width),
  batch_size=32,
  seed=42,
  shuffle=True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/drive/MyDrive/1주일 프로젝트/Data_set',
  validation_split=0.2,
  subset='validation',
  image_size=(img_height, img_width),
  batch_size=32,
  seed=42,
  shuffle=True)

Found 1218 files belonging to 3 classes.
Using 975 files for training.
Found 1218 files belonging to 3 classes.
Using 243 files for validation.


In [10]:
# 훈련 데이터셋 클래스 이름 확인 (Auto)

class_names = train_ds.class_names
print(class_names)
train_ds

['Cleaning', 'Exchange', 'Normal']


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 244, 244, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
# 훈련 데이터셋 샘플 이미지

plt.figure(figsize=(15, 15))
for images, labels in train_ds.take(1):
    for i in range(25):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
#  모델 로드 및 설정 (가중치 유지)

base_model = EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=(img_height, img_width, 3)
)
base_model.trainable = False


In [ ]:
# 커스텀 모델 아키텍처 정의
inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# 모델 컴파일
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

In [ ]:
# 모델 컴파일 및 학습 설정

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# 모델 훈련

epoch = 15
model.fit(train_ds, validation_data=val_ds, epochs=epoch,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=0.01,
            patience=3,
            verbose=1,
            restore_best_weights=True
        )
    ]
)

In [ ]:
# 모델의 미세 조정 설정 및 요약

base_model.trainable = True
for layer in base_model.layers[:14]:
    layer.trainable = False
model.summary()

In [ ]:
# 미세 조정을 위한 모델 재컴파일

model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# 모델 훈련 (미세 조정)
epoch = 15
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epoch,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=0.01,
            patience=3,
            verbose=1,
        )
    ]
)

# 훈련이 끝난 후 모델 저장
model.save('/content/drive/MyDrive/1주일 프로젝트/PARK/EfficientNetB3_model/EfficientNetB3_model.h5')

# 시각화

In [ ]:
# 훈련 및 검증 데이터에 대한 정확도와 손실

get_ac = history.history['accuracy']
get_los = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(len(get_ac))
plt.plot(epochs, get_ac, 'g', label='Accuracy of Training data')
plt.plot(epochs, get_los, 'r', label='Loss of Training data')
plt.title('Training data accuracy and loss')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, get_ac, 'g', label='Accuracy of Training Data')
plt.plot(epochs, val_acc, 'r', label='Accuracy of Validation Data')
plt.title('Training and Validation Accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, get_los, 'g', label='Loss of Training Data')
plt.plot(epochs, val_loss, 'r', label='Loss of Validation Data')
plt.title('Training and Validation Loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
# 모델 평가 및 예측 결과 시각화

loss, accuracy = model.evaluate(val_ds)

plt.figure(figsize=(20, 20))
for images, labels in val_ds.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predictions = model.predict(tf.expand_dims(images[i], 0))
        score = tf.nn.softmax(predictions[0])
        if(class_names[labels[i]]==class_names[np.argmax(score)]):
            plt.title("Actual: "+class_names[labels[i]])
            plt.ylabel("Predicted: "+class_names[np.argmax(score)],fontdict={'color':'green'})

        else:
            plt.title("Actual: "+class_names[labels[i]])
            plt.ylabel("Predicted: "+class_names[np.argmax(score)],fontdict={'color':'red'})
        plt.gca().axes.yaxis.set_ticklabels([])
        plt.gca().axes.xaxis.set_ticklabels([])
plt.show()

# 저장된 모델 로드 후 검증

In [ ]:
# 저장된 모델 불러오기
loaded_model = load_model('/content/drive/MyDrive/1주일 프로젝트/PARK/EfficientNetB3_model/EfficientNetB3_model.h5')

In [ ]:
# 원본 모델로 검증 데이터셋 평가
loss, accuracy = loaded_model.evaluate(val_ds)

In [ ]:
# 저장된 최종 모델 시각화

plt.figure(figsize=(20, 20))
for images, labels in val_ds.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        # 저장된 모델로 예측 수행
        predictions = loaded_model.predict(tf.expand_dims(images[i], 0))
        score = tf.nn.softmax(predictions[0])
        if(class_names[labels[i]] == class_names[np.argmax(score)]):
            plt.title("Actual: " + class_names[labels[i]])
            plt.ylabel("Predicted: " + class_names[np.argmax(score)], fontdict={'color': 'green'})
        else:
            plt.title("Actual: " + class_names[labels[i]])
            plt.ylabel("Predicted: " + class_names[np.argmax(score)], fontdict={'color': 'red'})
        plt.gca().axes.yaxis.set_ticklabels([])
        plt.gca().axes.xaxis.set_ticklabels([])
plt.show()

# 실제 Test 진행

In [ ]:
model_path = '/content/drive/MyDrive/1주일 프로젝트/PARK/EfficientNetB3_model/EfficientNetB3_model.h5'
model = load_model(model_path)

In [ ]:
import os
from tensorflow.keras.preprocessing import image

# 테스트 이미지가 있는 경로
test_images_directory = '/content/drive/MyDrive/1주일 프로젝트/Test_set'

# 훈련 데이터셋 클래스 이름 확인
class_names = train_ds.class_names
print(class_names)

# 모든 이미지 파일 확장자
image_formats = ['jpg', 'jpeg', 'png', 'bmp', 'tiff']

# 모든 하위 디렉토리를 탐색하여 이미지 파일 찾기
test_images = []
for fmt in image_formats:
    test_images.extend(glob.glob(test_images_directory + '/**/*.' + fmt, recursive=True))

plt.figure(figsize=(20, 20))

# 첫 16개 이미지에 대해 예측 수행
for i in range(9):
    img_path = test_images[i]
    img = image.load_img(img_path, target_size=(244, 244))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)

    # 예측 결과 시각화
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(img)
    plt.title("Actual: " + os.path.basename(os.path.dirname(img_path)))
    plt.ylabel("Predicted: " + class_names[np.argmax(classes)], fontdict={'color': 'green' if class_names[np.argmax(classes)] in os.path.basename(os.path.dirname(img_path)) else 'red'})
    plt.gca().axes.yaxis.set_ticklabels([])
    plt.gca().axes.xaxis.set_ticklabels([])
plt.show()

['Cleaning', 'Exchange', 'Normal']


AttributeError: 'function' object has no attribute 'glob'